### Deliverable 3: Optimize the model to get over 75% accuracy

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'], axis=1)

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than 500?
replace_application = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than 1800?
replace_class = list(classification_counts[classification_counts < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes=='object'].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### **Original Model**



In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 352       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=30)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/30
804/804 [==============================] - 2s 1ms/step - loss: 0.6683 - accuracy: 0.6217
Epoch 2/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5789 - accuracy: 0.7152
Epoch 3/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5764 - accuracy: 0.7168
Epoch 4/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5589 - accuracy: 0.7308
Epoch 5/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5555 - accuracy: 0.7317
Epoch 6/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5592 - accuracy: 0.7286
Epoch 7/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5577 - accuracy: 0.7299
Epoch 8/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5511 - accuracy: 0.7333
Epoch 9/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5568 - accuracy: 0.7280
Epoch 10/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5479 - accuracy: 0.7355

### **Optimzation Attempt 1:** Add a third and fourth hidden layer

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 6
hidden_nodes_layer3 = 4
hidden_nodes_layer4 = 2

nn_opt1 = tf.keras.models.Sequential()

# First hidden layer
nn_opt1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_opt1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_opt1.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn_opt1.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn_opt1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 352       
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 28        
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 3         
Total params: 447
Trainable params: 447
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn_opt1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_opt1 = nn_opt1.fit(X_train_scaled,y_train,epochs=30)

# Evaluate the model using the test data
model_loss_opt1, model_accuracy_opt1 = nn_opt1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_opt1}, Accuracy: {model_accuracy_opt1}")

Epoch 1/30
804/804 [==============================] - 2s 1ms/step - loss: 0.6528 - accuracy: 0.6012
Epoch 2/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5714 - accuracy: 0.7180
Epoch 3/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5576 - accuracy: 0.7283
Epoch 4/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5522 - accuracy: 0.7287
Epoch 5/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5532 - accuracy: 0.7294
Epoch 6/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5562 - accuracy: 0.7232
Epoch 7/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7336
Epoch 8/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5500 - accuracy: 0.7283
Epoch 9/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5496 - accuracy: 0.7307
Epoch 10/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7306

### **Optimzation Attempt 2:** Add neurons to hidden layers 

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  50
hidden_nodes_layer2 = 25

nn_opt2 = tf.keras.models.Sequential()

# First hidden layer
nn_opt2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_opt2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_opt2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 50)                2200      
_________________________________________________________________
dense_9 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 26        
Total params: 3,501
Trainable params: 3,501
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn_opt2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_opt2 = nn_opt2.fit(X_train_scaled,y_train,epochs=30)

# Evaluate the model using the test data
model_loss_opt2, model_accuracy_opt2 = nn_opt2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_opt2}, Accuracy: {model_accuracy_opt2}")

Epoch 1/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5938 - accuracy: 0.7011
Epoch 2/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5604 - accuracy: 0.7278
Epoch 3/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5532 - accuracy: 0.7292
Epoch 4/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5452 - accuracy: 0.7339
Epoch 5/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5469 - accuracy: 0.7311
Epoch 6/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5482 - accuracy: 0.7311
Epoch 7/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7297
Epoch 8/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5472 - accuracy: 0.7344
Epoch 9/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7253
Epoch 10/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5441 - accuracy: 0.7389

### **Optimzation Attempt 3:** Change Activation Function 

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_opt3 = tf.keras.models.Sequential()

# First hidden layer
nn_opt3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn_opt3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn_opt3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 8)                 352       
_________________________________________________________________
dense_12 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 6         
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn_opt3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_opt3 = nn_opt3.fit(X_train_scaled,y_train,epochs=30)

# Evaluate the model using the test data
model_loss_opt3, model_accuracy_opt3 = nn_opt3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_opt3}, Accuracy: {model_accuracy_opt3}")

Epoch 1/30
804/804 [==============================] - 2s 1ms/step - loss: 0.6611 - accuracy: 0.6040
Epoch 2/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5721 - accuracy: 0.7241
Epoch 3/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5609 - accuracy: 0.7314
Epoch 4/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5575 - accuracy: 0.7311
Epoch 5/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5606 - accuracy: 0.7276
Epoch 6/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5548 - accuracy: 0.7343
Epoch 7/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5579 - accuracy: 0.7299
Epoch 8/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5540 - accuracy: 0.7314
Epoch 9/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5509 - accuracy: 0.7353
Epoch 10/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5530 - accuracy: 0.7317

### **Optimzation Attempt 4:** Adjust Input Data (Drop INCOME_AMT)

In [19]:
application_opt4_df = application_df.drop(['INCOME_AMT_0', 'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999', 
                                      'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M', 
                                      'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M'],1)

In [20]:
# Split our preprocessed data into our features and target arrays
y_opt4 = application_opt4_df["IS_SUCCESSFUL"].values
X_opt4 = application_opt4_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train_opt4, X_test_opt4, y_train_opt4, y_test_opt4 = train_test_split(X_opt4, y_opt4, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler_opt4 = scaler.fit(X_train_opt4)

# Scale the data
X_train_scaled_opt4 = X_scaler_opt4.transform(X_train_opt4)
X_test_scaled_opt4 = X_scaler_opt4.transform(X_test_opt4)

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_opt4[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_opt4 = tf.keras.models.Sequential()

# First hidden layer
nn_opt4.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_opt4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_opt4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt4.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 8)                 280       
_________________________________________________________________
dense_15 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 6         
Total params: 331
Trainable params: 331
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn_opt4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_opt4 = nn_opt4.fit(X_train_scaled_opt4,y_train_opt4,epochs=30)

# Evaluate the model using the test data
model_loss_opt4, model_accuracy_opt4 = nn_opt4.evaluate(X_test_scaled_opt4,y_test_opt4,verbose=2)
print(f"Loss: {model_loss_opt4}, Accuracy: {model_accuracy_opt4}")

Epoch 1/30
804/804 [==============================] - 2s 1ms/step - loss: 0.7184 - accuracy: 0.5869
Epoch 2/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5953 - accuracy: 0.7145
Epoch 3/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5779 - accuracy: 0.7259
Epoch 4/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5705 - accuracy: 0.7251
Epoch 5/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5688 - accuracy: 0.7249
Epoch 6/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5648 - accuracy: 0.7277
Epoch 7/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5643 - accuracy: 0.7253
Epoch 8/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5613 - accuracy: 0.7283
Epoch 9/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5643 - accuracy: 0.7265
Epoch 10/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5603 - accuracy: 0.7297

### **Optimzation Attempt 5:** Combine optimizations 1-4 together

In [25]:
application_opt5_df = application_df.drop(['INCOME_AMT_0', 'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999', 
                                      'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M', 
                                      'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M'],1)

In [26]:
# Split our preprocessed data into our features and target arrays
y_opt5 = application_opt5_df["IS_SUCCESSFUL"].values
X_opt5 = application_opt5_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train_opt5, X_test_opt5, y_train_opt5, y_test_opt5 = train_test_split(X_opt5, y_opt5, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler_opt5 = scaler.fit(X_train_opt5)

# Scale the data
X_train_scaled_opt5 = X_scaler_opt5.transform(X_train_opt5)
X_test_scaled_opt5 = X_scaler_opt5.transform(X_test_opt5)

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_opt5[0])
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 20
hidden_nodes_layer4 = 10

nn_opt5 = tf.keras.models.Sequential()

# First hidden layer
nn_opt5.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn_opt5.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn_opt5.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Fourth hidden layer
nn_opt5.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn_opt5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt5.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 50)                1750      
_________________________________________________________________
dense_33 (Dense)             (None, 30)                1530      
_________________________________________________________________
dense_34 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_35 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 11        
Total params: 4,121
Trainable params: 4,121
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn_opt5.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_opt5 = nn_opt5.fit(X_train_scaled_opt5,y_train_opt5,epochs=30)

# Evaluate the model using the test data
model_loss_opt5, model_accuracy_opt5 = nn_opt5.evaluate(X_test_scaled_opt5,y_test_opt5,verbose=2)
print(f"Loss: {model_loss_opt5}, Accuracy: {model_accuracy_opt5}")

Epoch 1/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5863 - accuracy: 0.7111
Epoch 2/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5715 - accuracy: 0.7224
Epoch 3/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5636 - accuracy: 0.7259
Epoch 4/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5578 - accuracy: 0.7287
Epoch 5/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5558 - accuracy: 0.7323
Epoch 6/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5552 - accuracy: 0.7325
Epoch 7/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5549 - accuracy: 0.7301
Epoch 8/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5568 - accuracy: 0.7300
Epoch 9/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7265
Epoch 10/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5542 - accuracy: 0.7316

In [35]:
print(f"Original Model - Loss: {model_loss:,.4f}, Accuracy: {model_accuracy:,.4f}")
print(f"Optimization 1 - Loss: {model_loss_opt1:,.4f}, Accuracy: {model_accuracy_opt1:,.4f}")
print(f"Optimization 2 - Loss: {model_loss_opt2:,.4f}, Accuracy: {model_accuracy_opt2:,.4f}")
print(f"Optimization 3 - Loss: {model_loss_opt3:,.4f}, Accuracy: {model_accuracy_opt3:,.4f}")
print(f"Optimization 4 - Loss: {model_loss_opt4:,.4f}, Accuracy: {model_accuracy_opt4:,.4f}")
print(f"Optimization 5 - Loss: {model_loss_opt5:,.4f}, Accuracy: {model_accuracy_opt5:,.4f}")

Original Model - Loss: 0.5561, Accuracy: 0.7256
Optimization 1 - Loss: 0.5523, Accuracy: 0.7261
Optimization 2 - Loss: 0.5517, Accuracy: 0.7269
Optimization 3 - Loss: 0.5541, Accuracy: 0.7263
Optimization 4 - Loss: 0.5618, Accuracy: 0.7219
Optimization 5 - Loss: 0.5540, Accuracy: 0.7264
